In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import numpy as np

from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
Pipe = Pipeline(steps = [
    ("impute", SimpleImputer(strategy = "constant", fill_value=0)),
    ("MyTransformer", MyTransformer(df))
])

In [ ]:
X = Pipe.fit_transform(df)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class MyTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, df):
        self.df = df
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, df):
        X_transformed = X.tolist()
        df['area'] = df['area'].str.replace(' m²','')
        return df

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep

class WebScraper:

    def __init__(self, page):
        self.page = page
        self.tmp = []
        self.ListOfScrapedFlatInfo = []
    def return_html(self):
        # html_file = open(self.page, "r", encoding="utf-8")
        html_file = requests.get(self.page)
        return_html = BeautifulSoup(html_file.text, "html.parser")
        return return_html

    def return_href(self):
        soup = self.return_html()
        self.tmp = [a["href"] for a in soup.find_all("a", href=True) if a.parent.name == 'h3']
        return [a["href"] for a in soup.find_all("a", href=True) if a.parent.name == 'h3']


class Aruodas(WebScraper):
    def loop(self):    	
        for flat in self.tmp:
            sleep(5)
            self.ListOfScrapedFlatInfo.append((self.ScrapeFlat(flat)))

    def ScrapeFlat(self, page):
        d = {}
        self.page = page
        d["Adresas"] = self.page 
        soup = self.return_html()
        dd = soup.select("dd")
        dt = soup.select("dt")
        for i in range(20):
            try:
                tmp_val = str(dd[i].getText()).strip()
                tmp_name = str(dt[i].getText()).strip()
                d[tmp_name] = tmp_val
            except:
                return d
        return d

aruodas = Aruodas("https://www.aruodas.lt/")
output = aruodas.ScrapeFlat("https://www.aruodas.lt/butu-nuoma-vilniuje-pasilaiciuose-leiciu-g-naujas-ir-kokybiskai-irengtas-butas-laukia-4-955759/")
df = pd.DataFrame()
df = df.append(output, ignore_index=True)
df

In [ ]:
df['Kaina mėn.:'] = df['Kaina mėn.:'].str.replace(' €','')
df['Kaina mėn.:'] = df['Kaina mėn.:'].str.replace(' ','')

df.rename(columns={"Aukštas:": "floor", "Aukštų sk.:": "total_floor_num",
                   "Kambarių sk.:":"room_num", "Metai:":"year_full",
                   "Plotas:":"area","Pastato tipas:":"house_type", 
                  "Šildymas:":"heating", "Apsauga:": "security",
                   "Ypatybės:":"general"
                  }, inplace=True)
df

In [ ]:
try:
    df[['year','year_renovation']] = df['year_full'].str.split(',',expand=True)
    df['year'] = df['year_full'].str[:4]
    df['year'].fillna(df["year_full"],inplace = True)
    df["year"] = pd.to_numeric(df["year"],downcast='integer')
    df['year_renovation'] = df['year_renovation'].str[:5]
    df["year_renovation"] = pd.to_numeric(df["year_renovation"],downcast='integer')
except:
    df['year'] = df['year_full'].str[:4]
    df['year_renovation'] = 0

df['area'] = df['area'].str.replace(' m²','')
df['area'] = df['area'].str.replace(',','.')

df["top_floor"] = (df["total_floor_num"] == df["floor"]).astype('int')
df.loc[df[df['Adresas'].str.contains("senamiestyje").values].index,"oldtown"] = 1
df["oldtown"].fillna(value = 0, inplace = True)


districts = ["pasilaiciuose", "zveryne", "pilaiteje", "senamiestyje", 
             "naujamiestyje", "uzupyje", "snipiskese", "antakalnyje",
            "zirmunuose","lazdyneliuose", "karoliniskese", "naujojoje-vilnioje", "tarandeje","seskineje",
             "grigiskese", "santariskese", "balsiuose", "siaures-miestelyje",
             "markuciuose", "virsuliskese","fabijoniskese", "valakampiuose",
             "baltupiuose","justiniskese","visoriuose","lazdynuose","jeruzaleje"
             ,"avizieniuose","filaretuose","kalnenuose",
             "pavilnyje","bajoruose", "burbiskes","naujininkuose","vilkpedeje",
             "zemuosiuose-paneriuose","aukstuosiuose-paneriuose","jeruzaleje","sauletekyje","traku","verkiuose"
            ]

def map_district(district):    
    df.loc[df[df['Adresas'].str.contains(district).values].index, district] = 1
    df[district].fillna(value = 0, inplace = True)
    
for i in districts:
    map_district(i)

    
df

In [ ]:
COLUMNS = ['floor', 'total_floor_num', 'room_num', 'area', 'year',
       'year_renovation', 'top_floor', 'oldtown', '0', 'antakalnyje',
       'aukstuosiuose-paneriuose', 'avizieniuose', 'bajoruose', 'balsiuose',
       'baltupiuose', 'burbiskes', 'fabijoniskese', 'filaretuose',
       'grigiskese', 'jeruzaleje', 'justiniskese', 'kalnenuose',
       'karoliniskese', 'lazdyneliuose', 'lazdynuose', 'markuciuose',
       'naujamiestyje', 'naujininkuose', 'naujojoje-vilnioje', 'pasilaiciuose',
       'pavilnyje', 'pilaiteje', 'santariskese', 'sauletekyje', 'senamiestyje',
       'seskineje', 'siaures-miestelyje', 'snipiskese', 'tarandeje', 'traku',
       'uzupyje', 'valakampiuose', 'verkiuose', 'vilkpedeje', 'virsuliskese',
       'visoriuose', 'zemuosiuose-paneriuose', 'zirmunuose', 'zveryne']
prediction = pd.DataFrame(columns=columns)
prediction

In [ ]:
prediction = prediction.append(df)
prediction = prediction[columns]
prediction

In [ ]:
import joblib

model = joblib.load('model.pkl')

In [ ]:
model.predict(prediction)

In [14]:
COLUMNS = ['floor', 'total_floor_num', 'room_num', 'area', 'year',
       'year_renovation', 'top_floor', 'oldtown', '0', 'antakalnyje',
       'aukstuosiuose-paneriuose', 'avizieniuose', 'bajoruose', 'balsiuose',
       'baltupiuose', 'burbiskes', 'fabijoniskese', 'filaretuose',
       'grigiskese', 'jeruzaleje', 'justiniskese', 'kalnenuose',
       'karoliniskese', 'lazdyneliuose', 'lazdynuose', 'markuciuose',
       'naujamiestyje', 'naujininkuose', 'naujojoje-vilnioje', 'pasilaiciuose',
       'pavilnyje', 'pilaiteje', 'santariskese', 'sauletekyje', 'senamiestyje',
       'seskineje', 'siaures-miestelyje', 'snipiskese', 'tarandeje', 'traku',
       'uzupyje', 'valakampiuose', 'verkiuose', 'vilkpedeje', 'virsuliskese',
       'visoriuose', 'zemuosiuose-paneriuose', 'zirmunuose', 'zveryne']

PAGE = "https://www.aruodas.lt/butu-nuoma-vilniuje-fabijoniskese-salomejos-neries-g-isnuomojamas-dveju-kambariu-butas-esantis-4-955971/"

DICT_FOR_DF_RENAME = {"Aukštas:": "floor", "Aukštų sk.:": "total_floor_num",
                   "Kambarių sk.:":"room_num", "Metai:":"year_full",
                   "Plotas:":"area","Pastato tipas:":"house_type", 
                  "Šildymas:":"heating", "Apsauga:": "security",
                   "Ypatybės:":"general"
                  }

DISTRICTS = ["pasilaiciuose", "zveryne", "pilaiteje", "senamiestyje", 
             "naujamiestyje", "uzupyje", "snipiskese", "antakalnyje",
            "zirmunuose","lazdyneliuose", "karoliniskese", "naujojoje-vilnioje", "tarandeje","seskineje",
             "grigiskese", "santariskese", "balsiuose", "siaures-miestelyje",
             "markuciuose", "virsuliskese","fabijoniskese", "valakampiuose",
             "baltupiuose","justiniskese","visoriuose","lazdynuose","jeruzaleje"
             ,"avizieniuose","filaretuose","kalnenuose",
             "pavilnyje","bajoruose", "burbiskes","naujininkuose","vilkpedeje",
             "zemuosiuose-paneriuose","aukstuosiuose-paneriuose","jeruzaleje","sauletekyje","traku","verkiuose"
            ]


def ScrapeFlat(page):
    d = {}
    d["Adresas"] = page
    html_file = requests.get(page)
    soup = BeautifulSoup(html_file.text, "html.parser")
    dd = soup.select("dd")
    dt = soup.select("dt")
    for i in range(20):
        try:
            tmp_val = str(dd[i].getText()).strip()
            tmp_name = str(dt[i].getText()).strip()
            d[tmp_name] = tmp_val
        except:
            return d
    return d

def CreateDataFrameHeader(COLUMNS):    
    return pd.DataFrame(columns=COLUMNS)



def FromDictToDF(d):
    dfFlat = pd.DataFrame()
    dfFlat = dfFlat.append(d, ignore_index=True)
    return dfFlat

def CleanDF(dfFlat):    
    dfFlat = dfFlat.rename(columns={"Aukštas:": "floor", "Aukštų sk.:": "total_floor_num",
                   "Kambarių sk.:":"room_num", "Metai:":"year_full",
                   "Plotas:":"area","Pastato tipas:":"house_type", 
                  "Šildymas:":"heating", "Apsauga:": "security",
                   "Ypatybės:":"general"})    
    try:
        dfFlat[['year','year_renovation']] = dfFlat['year_full'].str.split(',',expand=True)
        dfFlat['year'] = dfFlat['year_full'].str[:4]
        dfFlat['year'].fillna(dfFlat["year_full"],inplace = True)
        dfFlat["year"] = pd.to_numeric(dfFlat["year"],downcast='integer')
        dfFlat['year_renovation'] = dfFlat['year_renovation'].str[:5]
        dfFlat["year_renovation"] = pd.to_numeric(dfFlat["year_renovation"],downcast='integer')
    except:
        dfFlat['year'] = dfFlat['year_full'].str[:4]
        dfFlat['year_renovation'] = 0
        
    dfFlat['area'] = dfFlat['area'].str.replace(' m²','')
    dfFlat['area'] = dfFlat['area'].str.replace(',','.')
    return dfFlat

def map_district(district):    
    dfFlat.loc[dfFlat[dfFlat['Adresas'].str.contains(district).values].index, district] = 1
    dfFlat[district].fillna(value = 0, inplace = True)
    
d = ScrapeFlat(PAGE)
dfFlat = FromDictToDF(d)
dfFlat = CleanDF(dfFlat)
df = CreateDataFrameHeader(COLUMNS)

dfFlat = df.append(dfFlat, ignore_index=True)

for i in DISTRICTS:
    map_district(i)

dfFlat = dfFlat[COLUMNS]
dfFlat.fillna(value = 0, inplace = True)

C:\Users\Titas\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [15]:
import joblib
model = joblib.load('model.pkl')
model.predict(dfFlat)

C:\Users\Titas\Anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.21.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Titas\Anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.21.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Titas\Anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.21.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Titas\Anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator Pipeline from version 0.21.2 when using version 0.21.3. This might lead to breaking code or invalid results

array([388.99083333])